In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
import joblib
from stock_price_api import *

In [ ]:
model = load_model('lstm_model.h5')
scaler = joblib.load('scaler.gz')

In [ ]:
df : pd.DataFrame = format_data(data = get_data())

In [ ]:
print(df)

In [ ]:
def get_scaled_data(df: pd.DataFrame, scaler: MinMaxScaler, data_size:int=500, path:str=r"..\Data\AAPL_stock_prices.csv", delimeter: str = ',', from_end: bool = True, date_column: str = 'Date', target_column: str = 'Close'):
    target_column_index = df.columns.tolist().index(target_column)
    scaled_data = scaler.fit_transform(df)
    return scaled_data, target_column_index

In [ ]:
scaled_data, target_column_index = get_scaled_data(df=df, scaler=scaler)

In [ ]:
def create_dataset(data: np.ndarray, time_step: int=10):
    X = []
    for i in range(len(data) - time_step):
        # Define the range of input sequences
        end_ix = i + time_step
        
        # Define the range of output sequences
        out_end_ix = end_ix
        
        # Ensure that the dataset is within bounds
        if out_end_ix > len(data)-1:
            break
            
        # Extract input and output parts of the pattern
        seq_x = data[i:end_ix]
        
        # Append the parts
        X.append(seq_x)
    return np.array(X), data.shape[1], time_step

X, feature_number, time_step = create_dataset(data=scaled_data)
print(df)
print(X[:2])

In [ ]:
def log(predicted_data, X):
    from datetime import datetime
    current_datetime = datetime.now()
    current_datetime_str = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
    script = ""
    with open('log.txt', 'a') as file:s
        file.write(f"\n    Train R2 Score: {train_r2:.4f}, Test R2 Score: {test_r2:.4f}\n\n")

In [ ]:
predicted_data = model.predict(X)
print(predicted_data)

In [ ]:
# Inverse transform the predictions
def update_data_to_inverse(predicted_data: np.ndarray, scaler: MinMaxScaler, target_column_index: int, feature_number: int):
    new_dataset = np.zeros(shape=(len(predicted_data), feature_number))
    new_dataset[:,target_column_index] = predicted_data.flatten()
    return scaler.inverse_transform(new_dataset)[:, target_column_index].reshape(-1, 1)
predicted_data = update_data_to_inverse(predicted_data=predicted_data, scaler=scaler, target_column_index=target_column_index, feature_number=feature_number)
new_df = df[['Close']].iloc[-10:].copy()
new_df['Predicted_close'] = predicted_data

In [ ]:

next_day = df.index.max() + pd.DateOffset(days=1)
last_prediction = pd.DataFrame({'Close': [np.nan], 'Predicted_close': predicted_data[-1]}, index=[f"{next_day} 00:00:00"])
desired_prediction = np.full((11,1), np.nan)
new_df = pd.concat([new_df, last_prediction])
desired_prediction[-2] = new_df['Close'][-2]
desired_prediction[-1] = new_df['Predicted_close'][-1]
desired_prediction = desired_prediction.reshape(-1,1)
new_df['Desired_prediction'] = desired_prediction

In [ ]:

# Ensure the index is in datetime format
new_df.index = pd.to_datetime(new_df.index)

# Convert the datetime index to 'day-month' format
new_df.index = new_df.index.strftime('%m-%d')

print(new_df)

In [ ]:
plt.plot(new_df['Close'], label='Close')
plt.plot(new_df['Predicted_close'], label='Predicted Close')
plt.plot(new_df['Desired_prediction'], label='Desired Prediction')
plt.title('Predictons')
plt.ylabel('Prices')
plt.xlabel('Date')
plt.legend()
plt.show()